In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

os.environ['THEANO_FLAGS'] = "device=cuda0"

In [ ]:
import shelve
import pprint
import numpy as np
import lasagne
import theano
import theano.tensor as T
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from lproc import subset, rmap
import seqtools
from datasets import ch14dataset as dataset
from sltools.utils import gloss2seq, seq2gloss
from sltools.nn_utils import onehot, jaccard, compute_scores, make_nn_sequence_mapper
from sltools.postproc import optimize_boundaries, filter_longshort

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
from experiments.hmmvsrnn_reco.a_data import durations, gloss_seqs, tmpdir, \
   train_subset, val_subset, test_subset, vocabulary
from experiments.hmmvsrnn_reco.utils import autoreload_feats, reload_best_model

In [ ]:
experiment_name = "rnn_skel_tc3_180213"
report = shelve.open(os.path.join(tmpdir, experiment_name))
# model = "rnn"
# modality = "skel"
# variant = "tc3"
# date = "180213"
model = report['meta']['model']
modality = report['meta']['modality']
variant = report['meta']['variant']
date = report['meta']['date']

In [ ]:
feat_seqs = autoreload_feats(modality)

feat_seqs_train = [seqtools.gather(f, train_subset) for f in feat_seqs]
gloss_seqs_train = seqtools.gather(gloss_seqs, train_subset)
durations_train = subset(durations, train_subset)
targets_train = seqtools.smap(lambda g, d: gloss2seq(g, d, 0),
                              gloss_seqs_train, durations_train)

feat_seqs_val = [seqtools.gather(f, val_subset) for f in feat_seqs]
gloss_seqs_val = seqtools.gather(gloss_seqs, val_subset)
durations_val = seqtools.gather(durations, val_subset)
targets_val = seqtools.smap(lambda g, d: gloss2seq(g, d, 0), 
                            gloss_seqs_val, durations_val)

feat_seqs_test = [seqtools.gather(f, val_subset) for f in feat_seqs]
gloss_seqs_test = seqtools.gather(gloss_seqs, test_subset)
durations_test = seqtools.gather(durations, test_subset)
targets_test = seqtools.smap(lambda g, d: gloss2seq(g, d, 0),
                             gloss_seqs_test, durations_test)

# Training report

In [ ]:
all_batch_losses = []
all_epoch_losses = []
for i in sorted([e for e in report.keys() if e.startswith("epoch")]):
    r = report[i]
    all_batch_losses += r['batch_losses']
    all_epoch_losses.append(r['epoch_loss'])

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(np.arange(len(all_epoch_losses)), all_epoch_losses, c='red')
n_batches = len(all_batch_losses) // len(all_epoch_losses)
error = np.array([np.std(all_batch_losses[i:i+n_batches]) 
                  for i in range(0, len(all_batch_losses), n_batches)])
# plt.fill_between(np.arange(len(all_epoch_losses)), 
#                  np.maximum(0.00001, all_epoch_losses - error), 
#                  all_epoch_losses + error)
plt.yscale("log")
plt.show()
# plt.semilogy([10 ** (i - 5) for i in range(5)])

In [ ]:
best_epoch, model_dict, predict_fn = reload_best_model(report)

print("best epoch: {}".format(best_epoch))

epoch_report = report[best_epoch]
print(epoch_report['train_scores']['jaccard'])
print(epoch_report['train_scores']['framewise'])
print(epoch_report['val_scores']['jaccard'])
print(epoch_report['train_scores']['framewise'] - epoch_report['val_scores']['framewise'])

# Score

In [ ]:
predictions_train = [np.argmax(p, axis=1) for p in predict_fn(feat_seqs_train)]
ji_train, framewise_train, confusion_train = compute_scores(predictions_train, targets_train, vocabulary)

# predictions_val = [np.argmax(p, axis=1) for p in predict_fn(feats_seqs_val)]
# ji_val, framewise_val, confusion_val = compute_scores(predictions_val, targets_val, vocabulary)

# print("JI: {:.4f} / {:.4f}".format(ji_train, ji_val))
# print("Accuracy: {:.4f} / {:.4f}".format(framewise_train, framewise_val))

In [ ]:
cnames = [
    '∅','vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo']

cmap = matplotlib.cm.viridis
cmap.set_bad(cmap(0.001))

fig, ax1 = plt.subplots(1, 1, figsize=(6, 4))
im = ax1.matshow(confusion_train / np.sum(confusion_train, axis=1, keepdims=True), 
                 interpolation='none', cmap=cmap,
                 clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
ax1.set_yticks(np.arange(0, 22, 1))
ax1.set_xticks([])
ax1.set_yticklabels(cnames)

divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

fig, ax2 = plt.subplots(1, 1, figsize=(6, 4))
im = ax2.matshow(confusion_val / np.sum(confusion_val, axis=1, keepdims=True), 
                 interpolation='none', cmap='viridis',
                 clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
ax2.set_yticks(np.arange(0, 22, 1))
ax2.set_xticks([])
ax2.set_yticklabels(cnames)

divider = make_axes_locatable(ax2)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.show()

# Preview prediction

In [ ]:
def preview_seq(proba, ax=None):
    # 21 distinct colors
    
    cmap = np.array([[113,204,0], [209,73,251], [243,255,52], [223,119,255], 
         [139,255,150], [255,66,189], [1,222,201], [255,77,30], 
         [0,149,225], [137,106,0], [0,43,105], [255,230,180], 
         [111,0,66], [0,113,63], [251,177,255], [56,96,0], 
         [160,218,255], [74,0,6], [255,170,172], [0,62,95], 
         [93,43,0]]) / 255
    
    ax = ax or plt.gca()
    l = np.argmax(proba, axis=1)
    for g, start, stop in seq2gloss(l):
        start = max(0, start - 1)
        stop = min(len(proba), stop + 1)
        if g == 0:
            ax.plot(np.arange(start, stop), proba[start:stop, 0], ls=':', c=cmap[0])
        else:
            ax.plot(np.arange(start, stop), proba[start:stop, g], c=cmap[g])
            ax.fill_between(np.arange(start, stop),
                            0, proba[start:stop, g],
                            facecolor=cmap[g],
                            alpha=0.3)
    ax.set_ylim((0.1, 1.05))

In [ ]:
s = 62

proba = predict_fn([[fseq[s]] for fseq in feats_seqs_val])[0]
labels = onehot(gloss2seq(gloss_seqs_val[s], durations_val[s], 0), 
                np.arange(0, 21))

f = plt.figure(figsize=(13, 2))
ax = f.add_subplot(111)
preview_seq(proba[:], ax)
plt.title("model predictions")
plt.show()
f = plt.figure(figsize=(13, .7))
ax = f.add_subplot(111)
preview_seq(labels[:] * 1.0,  ax)
plt.title("targets")
plt.show()

# print(transformations[val_subset_augmented[s]])

# Analyse errors

In [ ]:
# Distribution of errors

scores = [jaccard(onehot(l, vocabulary), onehot(p, vocabulary))
         for l, p in zip(targets_val, predictions_val)]

plt.figure()
plt.hist(scores, np.linspace(0.0, 1, 40))
plt.title("Histogram of sequence-wise JI")
plt.show()

In [ ]:
# nb of false positives out of sequence vocabulary
np.mean([len(set(p_) - set(l_)) for p_, l_ in zip(predictions_val, targets_val)], axis=0)

In [ ]:
# confusion types
cum_err = np.sum(confusion_val, axis=1) - np.diag(confusion_val)
print("false pos: {}  false neg: {}, mis-class: {}".format(
    cum_err[0], np.sum(confusion_val[1:, 0]), np.sum(cum_err[1:]) - np.sum(confusion_val[1:, 0])))

In [ ]:
# correlate error with predicted gloss duration

plt.figure(figsize=(9, 4))

prediction_accuracy = [np.sum(l[start:stop] == g) 
            for p, l in zip(predictions_val, targets_val)
            for (g, start, stop) in seq2gloss(p)
            if g != 0]
none_accuracy = [np.sum(l[start:stop] == 0)
            for p, l in zip(predictions_val, targets_val)
            for (g, start, stop) in seq2gloss(p)
            if g != 0]
gloss_d = [stop - start
           for p in predictions_val 
           for (g, start, stop) in seq2gloss(p)
           if g != 0]

scores_pred = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
scores_none = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
total_d = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
for vp, vn, d in zip(prediction_accuracy, none_accuracy, gloss_d):
    idx = int(d / 5)
    scores_pred[idx] += vp
    scores_none[idx] += vn
    total_d[idx] += d

plt.gca().bar(np.arange(0, int(np.ceil(max(gloss_d) + 0.0001)), 5), 
              scores_pred / total_d,
              width=5,
              alpha=.5)
plt.gca().bar(np.arange(0, int(np.ceil(max(gloss_d) + 0.0001)), 5), 
              scores_none / total_d,
              width=5,
              alpha=.5)

plt.legend(["predicted class", "non-gesture class"])
plt.xlabel("subsequence duration (based on model prediction)")
plt.ylabel("accuracy")
plt.show()

# Score

In [ ]:
# Optimize duration filter

boundaries = optimize_boundaries(targets_val, predictions_val, vocabulary, (30, 100, 301))
print("Optimal range: ", boundaries)
print("FYI the score without is: {:.4f}".format(ji_val))

In [ ]:
# Validation score

ji_filtered_val, accuracy_filtered_val, confusion_filtered_val = compute_scores(
    [filter_longshort(p, boundaries, 0) for p in predictions_val], 
    targets_val, vocabulary)
print("validation score: {:.4f}".format(ji_filtered_val))

In [ ]:
# Test score

predictions_test = [np.argmax(p, axis=1) for p in predict_fn(feats_seqs_test)]

ji_filtered_test, accuracy_filtered_test, confusion_filtered_test = compute_scores(
    [filter_longshort(p, boundaries, 0) for p in predictions_test], 
    targets_test, vocabulary)
print("testing score: {:.4f}".format(ji_filtered_test))

In [ ]:
# Recap:

print("JI: {:.4f} / {:.4f}".format(ji_train, ji_val))
print("Accuracy: {:.4f} / {:.4f}".format(framewise_train, framewise_val))

recap = {
    "experiment_name": experiment_name,
    "best_epoch": best_epoch,
    "accuracy": (framewise_train, framewise_val, None),
    "ji": (ji_train, ji_val, None),
    "confusion": (confusion_train, confusion_val, None),
    "accuracy_filtered": (None, accuracy_filtered_val, accuracy_filtered_test),
    "ji_filtered": (None, ji_filtered_val, ji_filtered_test),
    "confusion_filtered": (None, confusion_filtered_val, None),
}
report['analysis'] = recap

# Analyse model

## Filters

In [ ]:
# from sklearn.manifold import TSNE
# from sltools.tconv import TemporalConv

# tc_l = None
# layers = lasagne.layers.get_all_layers(model_dict['l_linout'])

# for l in layers:
#     if isinstance(l, TemporalConv):
#         tc_l = l
#         break

# W = np.asarray(tc_l.W.eval())
# tsne = TSNE(n_components=1, n_iter=5000, n_iter_without_progress=100, verbose=True)
# filter_order = np.argsort(tsne.fit_transform(W)[:, 0])